In [1]:
import pandas as pd

df = pd.read_csv('../data/kaggle_ds.csv')
df

,Unnamed: 0,DateTime,buyin,tourn_id,table,HandID,table_size,level,playing,seat,...,pot_turn,pot_river,ante,blinds_level,bet_pre,bet_flop,bet_turn,bet_river,result,balance
0,0,2020-06-07 18:15:09,$0.92+$0.08,2929450288,1,215051507105,3,1,3,1,...,30,30,0,0,0,0,0,0,gave up,0
1,1,2020-06-07 18:15:09,$0.92+$0.08,2929450288,1,215051507105,3,1,3,2,...,30,30,0,10,10,0,0,0,gave up,-10
2,2,2020-06-07 18:15:09,$0.92+$0.08,2929450288,1,215051507105,3,1,3,3,...,30,30,0,20,20,0,0,0,took chips,10
3,3,2020-06-07 18:15:23,$0.92+$0.08,2929450288,1,215051520234,3,1,3,1,...,180,180,0,20,60,60,0,0,took chips,60
4,4,2020-06-07 18:15:23,$0.92+$0.08,2929450288,1,215051520234,3,1,3,2,...,180,180,0,0,0,0,0,0,gave up,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102610,102610,2021-03-30 19:00:02,$0.92+$0.08,3162349514,1,225348760558,3,1,3,2,...,560,560,0,10,500,0,0,0,took chips,60
102611,102611,2021-03-30 19:00:02,$0.92+$0.08,3162349514,1,225348760558,3,1,3,3,...,560,560,0,20,20,0,0,0,gave up,-20
102612,102612,2021-03-30 19:00:25,$0.92+$0.08,3162349514,1,225348768503,3,1,3,1,...,1030,1030,0,20,20,50,390,0,lost,-460
102613,102613,2021-03-30 19:00:25,$0.92+$0.08,3162349514,1,225348768503,3,1,3,2,...,1030,1030,0,0,20,50,490,0,won,470


In [2]:
# df.groupby(['HandID', 'UserID']).sum()
# df['UserID']

# Tables we can fill:
- Users
- Room
- Transaction
- Skin
- Inventory

## Transactions
- TransactionID
- SenderID
- ReceiverID
- Amount
- DateTime
- Description

In [6]:
import numpy as np

potID = 'TEXAS_HOLDEM'
df_transactions = df[['UserID', 'balance', 'DateTime', 'HandID']].copy()
df_transactions = df_transactions.drop(df_transactions[df_transactions['balance'] == 0].index)
df_transactions['TransactionID'] = df_transactions.index
df_transactions['SenderID'] = np.where(df_transactions['balance'] < 0, df_transactions['UserID'], potID)
df_transactions['ReceiverID'] = np.where(df_transactions['balance'] > 0, df_transactions['UserID'], potID)
df_transactions['Description'] = None

# df_transactions
df_transactions.to_csv('../data/transactions.csv')

## Skin & Room
### Skin
- SkinID
- Image (path)
- Description

### Room
- RoomID
- Log (path)
- ChatLog (path)

In [16]:
STORAGE_DIR = '../data/imgs/'
df_skins = pd.DataFrame({
    'SkinID' : [str(i) for i in range(5)],
    'Image' : [f'{STORAGE_DIR}skin{i}.jpg' for i in range(5)],
    'Description' : [
        'secret',
        'default',
        'red',
        'blue',
        'green'
    ]
})
df_skins.to_csv('../data/skins.csv')

In [17]:
LOGS_DIR = '../data/tmp/'
num_rooms = 4
df_rooms = pd.DataFrame({
    'RoomID' : [str(i) for i in range(num_rooms)],
    'Log' : [f'{LOGS_DIR}log{i}.txt' for i in range(num_rooms)],
    'ChatLog' : [f'{LOGS_DIR}chatlog{i}.txt' for i in range(num_rooms)]
})
df_rooms.to_csv('../data/rooms.csv')

## Users
- UserID (add `TEXAS_HOLDEM` id)
- Pass
- Balance
- CurrentSkin
- RoomID

In [52]:

import random, string

df_users = df_transactions[['UserID', 'balance']].groupby(['UserID']).sum().reset_index().copy()
df_users['CurrentSkin'] = [random.choices(df_skins['SkinID'], [0.005, .695, .1, .1, .1])[0] for i in range(len(df_users))]
df_users['RoomID'] = [str(i % 4) if i < 16 else None for i in range(len(df_users))]
def random_str(size): 
    return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=size))
df_users['Pass'] = [random_str(random.randrange(8, 15)) for i in range(len(df_users))]
df_users = df_users.rename(columns={'balance' : 'Balance'})
df_users.to_csv('../data/users.csv')